In [15]:
from V6_tratamiento import tratamiento, tratamiento2
from utils import *

import pandas as pd
import numpy as np

In [16]:
# Antes de nada veo si el modelo anterior mejora con las columnas

In [17]:
df_modelado = pd.read_pickle("./pickles_temp/V6.1.pkl") 
df_modelado.head(1)

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_class_groundwater,source_class_surface,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group,random1,random2
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,...,True,False,True,False,False,False,False,2,719,788


In [18]:
columnas_buenas = ['id', 'amount_tsh', 'amount_tsh_bool', 'funder',
       'funder_frequency_encoded', 'gps_height', 'installer',
       'installer_frequency_encoded', 'longitude', 'latitude', 'wpt_name',
       'wpt_name_frequency_encoded', 'num_private_b', 'basin', 'subvillage',
       'subvillage_frequency_encoded', 'region', 'region_frequency_encoded',
       'region_code', 'district_code', 'lga', 'lga_frequency_encoded', 'ward',
       'ward_frequency_encoded', 'population', 'public_meeting', 'permit',
       'construction_year', 'year_recorded', 'management_Parastatal',
       'management_Private operator', 'management_VWC', 'management_WUA',
       'management_WUG', 'management_Water Board',
       'management_Water authority', 'extraction_type_class_motorpump',
       'extraction_type_class_other', 'extraction_type_class_rope pump',
       'extraction_type_class_submersible',
       'extraction_type_class_wind-powered', 'management_group_other/unknown',
       'management_group_user-group', 'payment_type_monthly',
       'payment_type_on failure', 'payment_type_other/unknown',
       'payment_type_per bucket', 'quality_group_fluoride',
       'quality_group_milky', 'quality_group_salty', 'quality_group_unknown',
       'quantity_enough', 'quantity_insufficient', 'quantity_seasonal',
       'source_type_dam', 'source_type_other',
       'source_type_rainwater harvesting', 'source_type_river/lake',
       'source_type_shallow well', 'source_type_spring',
       'waterpoint_type_communal standpipe multiple',
       'waterpoint_type_improved spring', 'waterpoint_type_other', 'random1',
       'random2']

In [19]:
print(df_modelado.shape)
df_modelado = df_modelado[columnas_buenas]
print(df_modelado.shape)

(59400, 81)
(59400, 65)


In [20]:
# Añado status_group

training_set_values = pd.read_csv("training set labels.csv")

# Fusionar los DataFrames en base a la columna 'id'
df_modelado = pd.merge(df_modelado, training_set_values, how='inner', on='id')
df_modelado

,id,amount_tsh,amount_tsh_bool,funder,funder_frequency_encoded,gps_height,installer,installer_frequency_encoded,longitude,latitude,...,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_other,random1,random2,status_group
0,69572,6000.0,1,3376,275,1390.0,3477,94,34.938093,-9.856322,...,False,False,False,True,False,False,False,719,788,functional
1,8776,0.0,0,1709,53,1399.0,1750,39,34.698766,-2.147466,...,True,False,False,False,False,False,False,384,966,functional
2,34310,25.0,1,2413,8,686.0,4397,408,37.460664,-3.821329,...,False,False,False,False,True,False,False,770,718,functional
3,67743,0.0,0,3962,1057,263.0,4002,222,38.486161,-11.155298,...,False,False,False,False,True,False,False,571,16,non functional
4,19728,0.0,0,794,1,1527.0,906,135,31.130847,-1.825359,...,True,False,False,False,False,False,False,591,472,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,1,1663,610,1210.0,1093,610,37.169807,-3.253847,...,False,False,False,True,False,False,False,608,769,functional
59396,27263,4700.0,1,1118,23,1212.0,1184,15,35.249991,-9.070629,...,False,True,False,False,False,False,False,128,382,functional
59397,37057,0.0,0,667,1,1532.5,711,1,34.017087,-8.750434,...,False,False,False,False,False,False,False,526,296,functional
59398,31282,0.0,0,2535,3,820.5,2852,5,35.861315,-6.378573,...,False,False,True,False,False,False,False,558,22,functional


In [23]:
df_modelado['status_group'].replace(['non functional', 'functional needs repair', 'functional'], [0, 1, 2], inplace=True)

In [24]:
model_setup = setup(data=df_modelado, 
                    target = 'status_group', 
                    )

,Description,Value
0,Session id,3220
1,Target,status_group
2,Target type,Multiclass
3,Original data shape,"(59400, 66)"
4,Transformed data shape,"(59400, 66)"
5,Transformed train set shape,"(41580, 66)"
6,Transformed test set shape,"(17820, 66)"
7,Numeric features,29
8,Preprocess,True
9,Imputation type,simple


In [25]:
best_model = compare_models()
# Antes 0.8055
# Ahora 0.7977

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7977,0.8984,0.7977,0.7913,0.7865,0.6161,0.6220,2.6110
et,Extra Trees Classifier,0.7955,0.8983,0.7955,0.7879,0.7880,0.6167,0.6200,1.9970
xgboost,Extreme Gradient Boosting,0.7901,0.8929,0.7901,0.7855,0.7772,0.5989,0.6077,1.2650
lightgbm,Light Gradient Boosting Machine,0.7843,0.8883,0.7843,0.7843,0.7693,0.5842,0.5970,1.2800
gbc,Gradient Boosting Classifier,0.7543,0.8552,0.7543,0.7573,0.7323,0.5185,0.5400,14.8700
ridge,Ridge Classifier,0.7220,0.0000,0.7220,0.6834,0.6872,0.4453,0.4739,0.1570
lda,Linear Discriminant Analysis,0.7216,0.8125,0.7216,0.7187,0.6977,0.4524,0.4780,0.5890
ada,Ada Boost Classifier,0.7164,0.7976,0.7164,0.7085,0.6896,0.4427,0.4601,1.6600
dt,Decision Tree Classifier,0.7103,0.7441,0.7103,0.7121,0.7112,0.4774,0.4775,0.4310
lr,Logistic Regression,0.5972,0.6530,0.5972,0.5492,0.5593,0.1945,0.2082,3.8000


In [27]:
corr = abs(df_modelado.corr())
corr[['status_group']].sort_values(by = 'status_group',ascending = False).style.background_gradient()

""" Principales correladores en el V6.1

quantity_dry	0.398665
waterpoint_type_other	0.305463
extraction_type_class_other	0.295936
quantity_enough	0.248958
amount_tsh_bool	0.224277
construction_year	0.184744
payment_type_never pay	0.168765
waterpoint_type_communal standpipe	0.165314
quality_group_unknown	0.163276
ward_frequency_encoded	0.159053
region_frequency_encoded	0.151343 """
# Vemos que quantity_dry ha desaparecido probablemente por correlacionarse con quantity enough. Tan solo esa. 
# El modelo tiene una predictivilidad muy similar con muchas menos columnas. 
# En un concurso se trataria me tener el mejor score
# En una empresa de explicarlo
# Este es acadamico y busco probar cosas para aprender. Voy a seguir con él.


,status_group
status_group,1.000000
waterpoint_type_other,0.305463
extraction_type_class_other,0.295936
quantity_enough,0.248958
amount_tsh_bool,0.224277
construction_year,0.184744
quality_group_unknown,0.163276
ward_frequency_encoded,0.159053
region_frequency_encoded,0.151343
lga_frequency_encoded,0.136878


In [31]:
# Dado que tenemos una muestra desbalanceada quiero probar si el oversample mejora el modelo
df_modelado.to_pickle("./pickles_temp/V7.1.pkl")

(59400, 66)